<a href="https://colab.research.google.com/github/arockiaranjini/testsep1/blob/main/1_week_6_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [ ]:
words = open('/content/sample_data/names.txt','r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [ ]:
len(words)

32033

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
chars

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
# w -->letter in words
for w in words[:5]:
    print(w)
    #[0]--> .
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [ ]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [ ]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [ ]:
C = torch.randn(27,2)

In [ ]:
C

tensor([[-0.5985,  0.6246],
        [ 0.4146,  0.1480],
        [ 0.5193, -0.6891],
        [ 0.9772,  0.6164],
        [-0.1702,  1.7808],
        [ 0.9569,  1.2902],
        [-0.1899,  0.7296],
        [-0.1294, -0.6790],
        [ 0.8862,  0.4332],
        [-0.3611, -1.2462],
        [ 1.1567,  0.2232],
        [ 2.3348,  1.5768],
        [ 0.2252,  0.7303],
        [-0.7958, -0.7593],
        [ 0.0949,  0.9927],
        [-0.3943,  0.4676],
        [ 0.3658, -0.8441],
        [ 1.2469, -1.0275],
        [ 0.7224, -0.7977],
        [ 0.5345,  1.1042],
        [ 0.7009,  0.4909],
        [ 0.6348,  0.3719],
        [-0.8936, -2.1128],
        [-0.1074, -0.4924],
        [ 1.2452, -0.8715],
        [ 0.6264, -0.5913],
        [-0.3354,  0.9229]])

In [ ]:
C.shape

torch.Size([27, 2])

In [ ]:
C[5]

tensor([0.9569, 1.2902])

In [ ]:
F.one_hot(torch.tensor(5),num_classes = 27)

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [ ]:
emb = C[X]
emb

tensor([[[-0.5985,  0.6246],
         [-0.5985,  0.6246],
         [-0.5985,  0.6246]],

        [[-0.5985,  0.6246],
         [-0.5985,  0.6246],
         [ 0.9569,  1.2902]],

        [[-0.5985,  0.6246],
         [ 0.9569,  1.2902],
         [-0.7958, -0.7593]],

        [[ 0.9569,  1.2902],
         [-0.7958, -0.7593],
         [-0.7958, -0.7593]],

        [[-0.7958, -0.7593],
         [-0.7958, -0.7593],
         [ 0.4146,  0.1480]],

        [[-0.5985,  0.6246],
         [-0.5985,  0.6246],
         [-0.5985,  0.6246]],

        [[-0.5985,  0.6246],
         [-0.5985,  0.6246],
         [-0.3943,  0.4676]],

        [[-0.5985,  0.6246],
         [-0.3943,  0.4676],
         [ 0.2252,  0.7303]],

        [[-0.3943,  0.4676],
         [ 0.2252,  0.7303],
         [-0.3611, -1.2462]],

        [[ 0.2252,  0.7303],
         [-0.3611, -1.2462],
         [-0.8936, -2.1128]],

        [[-0.3611, -1.2462],
         [-0.8936, -2.1128],
         [-0.3611, -1.2462]],

        [[-0.8936, -2

In [ ]:
#alternative
F.one_hot(torch.tensor(5),num_classes = 27).float() @ C

tensor([0.9569, 1.2902])

In [ ]:
emb.shape

torch.Size([32, 3, 2])

In [ ]:
#hidden layer
W1 = torch.randn(6,100)
b1 = torch.randn(100)
# emb @ W1 + b1
# emb [32, 3, 2]  W1 [6, 100]
# [32, 100]


In [ ]:
W1

tensor([[-9.8183e-02,  4.9524e-01,  3.4335e-01,  1.6447e+00, -2.5268e-01,
          8.7134e-01, -8.3944e-01, -1.1137e+00, -2.7489e-01,  8.7768e-01,
         -1.7843e+00, -9.7738e-01,  2.5496e-01,  8.3042e-01, -1.3230e-01,
          3.7143e-01, -2.4177e-01,  8.1824e-02,  4.6145e-01,  1.1175e+00,
          1.1049e+00, -5.8938e-01,  1.4568e+00, -1.1017e+00,  1.7952e-01,
          1.0330e+00,  7.5859e-02,  5.2718e-01,  1.8077e+00,  2.7793e-01,
          1.3723e+00,  1.3244e+00, -1.1392e+00,  5.5220e-01, -1.5035e+00,
          1.8417e-01,  3.9946e-01, -3.5051e-01, -1.1690e+00, -1.0676e+00,
          1.6748e+00,  3.9256e-01,  1.7526e-01, -6.9495e-01,  4.2871e-01,
          1.1950e+00, -6.7204e-01, -1.6967e+00,  1.4218e+00,  1.0965e+00,
         -1.3164e+00,  3.1407e+00,  1.8581e+00,  8.7314e-01,  7.1773e-01,
          8.8150e-02,  7.4445e-01,  2.3654e+00,  5.5907e-02,  6.2556e-01,
         -3.7078e-01, -1.5716e+00, -9.2361e-01,  1.3199e-01,  3.9186e-01,
          2.0015e-01,  1.9470e+00, -7.

In [ ]:
emb.view(32,6)

tensor([[-0.5985,  0.6246, -0.5985,  0.6246, -0.5985,  0.6246],
        [-0.5985,  0.6246, -0.5985,  0.6246,  0.9569,  1.2902],
        [-0.5985,  0.6246,  0.9569,  1.2902, -0.7958, -0.7593],
        [ 0.9569,  1.2902, -0.7958, -0.7593, -0.7958, -0.7593],
        [-0.7958, -0.7593, -0.7958, -0.7593,  0.4146,  0.1480],
        [-0.5985,  0.6246, -0.5985,  0.6246, -0.5985,  0.6246],
        [-0.5985,  0.6246, -0.5985,  0.6246, -0.3943,  0.4676],
        [-0.5985,  0.6246, -0.3943,  0.4676,  0.2252,  0.7303],
        [-0.3943,  0.4676,  0.2252,  0.7303, -0.3611, -1.2462],
        [ 0.2252,  0.7303, -0.3611, -1.2462, -0.8936, -2.1128],
        [-0.3611, -1.2462, -0.8936, -2.1128, -0.3611, -1.2462],
        [-0.8936, -2.1128, -0.3611, -1.2462,  0.4146,  0.1480],
        [-0.5985,  0.6246, -0.5985,  0.6246, -0.5985,  0.6246],
        [-0.5985,  0.6246, -0.5985,  0.6246,  0.4146,  0.1480],
        [-0.5985,  0.6246,  0.4146,  0.1480, -0.8936, -2.1128],
        [ 0.4146,  0.1480, -0.8936, -2.1

In [ ]:
# 32,3,2 [-1 = 32]
h = emb.view(-1,6) @ W1 + b1

In [ ]:
#hidden layer
h.shape

torch.Size([32, 100])

In [ ]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [ ]:
h

tensor([[-0.9665,  0.1956, -0.8048,  ...,  0.5782,  0.9062, -0.6752],
        [-0.0653, -0.4874, -0.5023,  ..., -0.6743,  0.6397, -0.7071],
        [-0.7365, -0.9009, -0.9059,  ...,  0.9257,  0.7152, -1.0000],
        ...,
        [ 0.9579, -0.9973,  0.9775,  ...,  0.1087, -0.7537,  0.7352],
        [-0.9120, -0.9280, -0.9533,  ...,  0.9707,  0.5492, -0.9931],
        [ 0.5505, -0.9845,  0.9929,  ...,  0.9539, -0.1402,  0.9994]])

In [ ]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)
# h [32,100] W2 [100,27]
# [32,27]

In [ ]:
W2

tensor([[-1.1949,  1.7443,  0.6253,  ..., -1.5217, -1.8561,  0.3264],
        [ 0.3400,  0.9418, -0.0156,  ..., -1.4391, -0.0625,  0.0973],
        [-0.3136,  1.3849, -0.8615,  ...,  0.6549, -1.0142,  0.2038],
        ...,
        [ 1.4555,  0.8391,  0.7304,  ..., -0.8179, -0.8094,  0.6668],
        [ 1.1748,  0.2525, -1.1924,  ...,  0.8433,  1.4883,  0.5350],
        [ 0.8498,  1.6799,  0.7824,  ...,  0.8423,  2.8687,  0.4672]])

In [ ]:
logits = h @ W2 + b2

In [ ]:
h

tensor([[ 1.5589,  0.1688,  1.5206,  ...,  2.3221, -1.5223,  0.4649],
        [-1.7308,  0.7441,  0.2569,  ..., -1.0727, -3.7110,  0.9398],
        [-2.3198,  0.0481, -2.3803,  ..., -0.0709, -2.7397, -0.6533],
        ...,
        [-2.8118,  0.4140,  4.2234,  ...,  1.8294, -1.1517,  4.5967],
        [-2.6613,  0.1783,  1.0845,  ...,  1.3633, -5.7019,  1.1010],
        [ 3.7866, -0.8394,  1.2126,  ...,  4.9823,  1.0392,  0.8883]])

In [ ]:
logits.shape

torch.Size([32, 27])

In [ ]:
logits[0]

tensor([  9.5322,   0.0505,   4.9023, -11.6165, -12.6236,   7.9809,   0.6413,
          6.9113,   3.7647,  -1.5330,   5.9128,  -5.7028,   0.8669, -10.0738,
         -3.1963,   0.7728,   5.3305,  -8.0342,   5.1006,  -4.3876,   1.2677,
         -4.4247,  11.4761,  -0.0978,   1.3778, -11.2789,  -4.5994])

In [ ]:
prob = logits.exp() / logits.exp().sum(1, keepdims = True)

In [ ]:
prob[0]

tensor([1.1993e-01, 9.1425e-06, 1.1699e-03, 7.8371e-11, 2.8627e-11, 2.5420e-02,
        1.6505e-05, 8.7227e-03, 3.7508e-04, 1.8766e-06, 3.2138e-03, 2.9004e-08,
        2.0683e-05, 3.6655e-10, 3.5563e-07, 1.8826e-05, 1.7953e-03, 2.8177e-09,
        1.4266e-03, 1.0805e-07, 3.0879e-05, 1.0411e-07, 8.3781e-01, 7.8822e-06,
        3.4476e-05, 1.0984e-10, 8.7422e-08])

In [ ]:
prob[1].sum()

tensor(1.0000)

In [ ]:
prob

tensor([[1.1993e-01, 9.1425e-06, 1.1699e-03, 7.8371e-11, 2.8627e-11, 2.5420e-02,
         1.6505e-05, 8.7227e-03, 3.7508e-04, 1.8766e-06, 3.2138e-03, 2.9004e-08,
         2.0683e-05, 3.6655e-10, 3.5563e-07, 1.8826e-05, 1.7953e-03, 2.8177e-09,
         1.4266e-03, 1.0805e-07, 3.0879e-05, 1.0411e-07, 8.3781e-01, 7.8822e-06,
         3.4476e-05, 1.0984e-10, 8.7422e-08],
        [2.2796e-01, 1.1147e-08, 7.9202e-04, 5.9417e-07, 3.2427e-06, 1.1127e-04,
         2.7088e-04, 5.3273e-06, 8.9858e-04, 9.9334e-06, 7.3833e-01, 1.7596e-07,
         1.3613e-05, 2.6097e-13, 2.0725e-05, 6.1041e-09, 5.1622e-10, 2.5497e-05,
         5.0666e-03, 2.8994e-11, 3.1385e-05, 2.2053e-09, 2.6178e-02, 2.5831e-04,
         8.2250e-09, 5.7807e-09, 2.8066e-05],
        [1.6890e-03, 2.6506e-10, 2.7981e-08, 6.1133e-11, 4.7243e-03, 4.5906e-09,
         5.6345e-04, 2.9523e-04, 1.3969e-04, 6.9642e-05, 3.5882e-03, 2.0967e-08,
         4.7074e-08, 1.0202e-04, 2.5502e-03, 1.6577e-07, 6.0530e-03, 1.3328e-09,
         8.9628e-

In [ ]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [ ]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [ ]:
prob[torch.arange(32), Y]

tensor([2.5420e-02, 2.6097e-13, 1.0202e-04, 7.8527e-08, 2.1250e-12, 1.8826e-05,
        7.5376e-05, 5.0993e-05, 9.9372e-01, 4.9808e-08, 6.4269e-06, 2.3799e-15,
        9.1425e-06, 5.3728e-01, 1.5516e-12, 4.7916e-06, 1.8766e-06, 2.8884e-07,
        5.8920e-05, 4.8498e-03, 5.6223e-09, 1.1571e-12, 1.0129e-06, 9.1994e-14,
        9.7229e-01, 1.0805e-07, 2.1630e-08, 1.5744e-05, 8.0628e-09, 6.0679e-06,
        1.0281e-05, 2.0399e-04])

In [ ]:
-prob[torch.arange(32), Y].log().mean()

tensor(15.7704)

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100 , generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

3481

In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
prob = logits.exp() / logits.exp().sum(1, keepdims = True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)

In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss

tensor(17.7697)

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100 , generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

3481

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range(100):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

0.2789476215839386
0.27876269817352295
0.27858036756515503
0.2784004509449005
0.2782229483127594
0.2780477702617645
0.27787497639656067
0.27770447731018066
0.2775361239910126
0.27737003564834595
0.2772060036659241
0.2770441174507141
0.2768843173980713
0.27672645449638367
0.2765706479549408
0.2764166593551636
0.2762646973133087
0.2761145234107971
0.27596622705459595
0.27581968903541565
0.2756749987602234
0.27553197741508484
0.2753906548023224
0.27525097131729126
0.27511295676231384
0.27497658133506775
0.2748417556285858
0.27470850944519043
0.2745767831802368
0.274446576833725
0.27431780099868774
0.2741905152797699
0.2740646004676819
0.2739401161670685
0.2738170027732849
0.2736952602863312
0.2735748291015625
0.27345576882362366
0.27333787083625793
0.27322134375572205
0.2731059789657593
0.27299192547798157
0.27287906408309937
0.2727673351764679
0.27265676856040955
0.2725474238395691
0.2724391520023346
0.2723320424556732
0.2722259759902954
0.27212101221084595
0.27201706171035767
0.27191424

In [ ]:
# we are just talking about 5 words and 32 examples

In [ ]:
logits.max(1)

torch.return_types.max(
values=tensor([12.2507, 15.6167, 19.7251, 19.2065, 15.1653, 12.2507, 14.5368, 12.9834,
        14.6929, 16.7506, 14.3019, 19.3470, 12.2507, 15.2280, 15.6323, 18.4261,
        12.2507, 15.1299, 13.3647, 15.2784, 17.2229, 14.2677,  9.4187,  9.3573,
        14.6208, 12.2507, 14.8763, 15.4997, 11.8749, 15.2119, 17.4153, 14.0490],
       grad_fn=<MaxBackward0>),
indices=tensor([19, 13, 13,  1,  0, 19, 12,  9, 22,  9,  1,  0, 19, 22,  1,  0, 19, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0]))

In [ ]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [ ]:
# build the dataset
block_size = 3
X, Y = [], []
for w in words:
    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100 , generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

3481

In [ ]:
for p in parameters:
    p.requires_grad = True

19.505229949951172


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
for _ in range(100):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

5.803146839141846
5.722784042358398
5.64509391784668
5.5699462890625
5.497213840484619
5.4267802238464355
5.3585357666015625
5.2923760414123535
5.228203773498535
5.165928363800049
5.105468273162842
5.04674768447876
4.98969841003418
4.934261322021484
4.880379676818848
4.828005313873291
4.777095794677734
4.727609157562256
4.679513931274414
4.632779121398926
4.587378025054932
4.543288707733154
4.500490665435791
4.4589667320251465
4.418700218200684
4.379675388336182
4.3418779373168945
4.305292129516602
4.269898891448975
4.235679626464844
4.202611923217773
4.170670032501221
4.139823913574219
4.110044002532959
4.081292152404785
4.053531169891357
4.02672004699707
4.00081729888916
3.9757795333862305
3.951563596725464
3.9281277656555176
3.905430793762207
3.8834331035614014
3.862098217010498
3.84139084815979
3.821279764175415
3.8017334938049316
3.782726287841797
3.764232635498047
3.7462291717529297
3.7286949157714844
3.7116103172302246
3.6949586868286133
3.678722381591797
3.662886381149292
3.647

In [ ]:
# it will take a bit of time for each iteration

In [ ]:
X.shape

torch.Size([228146, 3])

In [ ]:
torch.randint(0,5,(32,))

tensor([0, 2, 0, 3, 4, 2, 4, 1, 1, 4, 0, 0, 2, 4, 0, 0, 1, 1, 3, 2, 1, 3, 3, 0,
        4, 2, 3, 3, 0, 3, 0, 2])

In [ ]:
C[X[ix]]

tensor([[[ 0.4216,  2.1558],
         [ 0.4216,  2.1558],
         [-0.0497, -0.3762]],

        [[-0.5696, -0.0213],
         [-0.6086,  0.0470],
         [-0.0497, -0.3762]],

        [[ 0.6119, -0.4498],
         [ 0.6119, -0.4498],
         [ 1.1269, -0.6584]],

        [[-0.5696, -0.0213],
         [-0.6086,  0.0470],
         [ 0.0873, -0.4394]],

        [[ 0.6119, -0.4498],
         [-0.6957, -0.8288],
         [ 0.4824,  2.1639]],

        [[-0.0363, -0.3810],
         [-0.4991, -0.0838],
         [-0.6086,  0.0470]],

        [[-0.6086,  0.0470],
         [-0.6086,  0.0470],
         [-0.0497, -0.3762]],

        [[ 0.0873, -0.4394],
         [-0.0363, -0.3810],
         [-0.4963, -0.0514]],

        [[-0.6086,  0.0470],
         [-0.0497, -0.3762],
         [-0.1314, -0.4111]],

        [[ 0.6119, -0.4498],
         [ 0.6119, -0.4498],
         [ 0.8065, -0.4813]],

        [[ 0.6119, -0.4498],
         [ 0.6119, -0.4498],
         [ 0.6119, -0.4498]],

        [[ 0.6119, -0

In [ ]:
ix = torch.randint(0,X.shape[0],(32,))
ix

tensor([134769, 144844,  67732,  78212, 224927, 158487, 120421, 175080,  41728,
         19708, 223992, 113349, 105881, 196089,  32317, 188719, 193024,  94020,
        226207, 128930, 155645,  23443, 154352,  37162, 114125, 155194,  22464,
         27565,  31664,  59863, 201293,  38038])

In [ ]:
#for _ in range(10):
for _ in range(1000):
    # mini batch construction
    ix = torch.randint(0,X.shape[0],(32,))

    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    #print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.00001 * p.grad

print(loss.item())

2.330946922302246


In [ ]:
# since we are doing mini batches, the quality of our gradient is lower
# its better to have approx gradient and take more steps
# rather than to evaluate exact gradient and take fewer steps

In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss


tensor(3.1985, grad_fn=<NllLossBackward0>)

In [ ]:
# this model is very small with close to 3k params, models can have millions or billions of param
# as the capacity of the neural network grows, it becomes more and more capable of overfitting the training set
# the loss on training set might become close to 0 and all what model is doing is memorizing all the examples it saw
# when we sample we get same names as the training set
# split the dataset into 3 splits (training, val/dev, test)
# 80% , 10%, 10%

In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100 , generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range(10000):

    # mini batch construction
    ix = torch.randint(0,Xtr.shape[0],(32,))

    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.01 * p.grad

print(loss.item())

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
# the loss on full training data and dev data are almost same, we are not overfitting.
# this model is not powerful enough to memorize things.
# that means our network is small, we can expect to make improvements by scaling up the size.

In [ ]:
#hidden layer bias 300
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,300), generator = g)
b1 = torch.randn(300 , generator = g)
W2 = torch.randn((300,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

10281

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range(10000):

    # mini batch construction
    ix = torch.randint(0,Xtr.shape[0],(32,))

    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.001 * p.grad

print(loss.item())

2.4627015590667725


In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.5742, grad_fn=<NllLossBackward0>)

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.5618, grad_fn=<NllLossBackward0>)

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

careah.
aal.
hlvihkimli.
reaty.
halassne.
rahnen.
amerart.
kaqui.
nelonia.
ceriiv.
kale.
gve.
bman.
cadesinn.
sroilea.
jamii.
wazero.
dearyni.
jameeniusdt.
edae.


In [ ]:
# since we have the embedding of 2, it might be a bottleneck

In [ ]:
# visualize dimensions 0 and 1 of the embedding matrix C for all characters
plt.figure(figsize=(10,10))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')
plt.show()

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,10), generator = g)
W1 = torch.randn((30,300), generator = g)
b1 = torch.randn(300 , generator = g)
W2 = torch.randn((300,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range(10000):

    # mini batch construction
    ix = torch.randint(0,Xtr.shape[0],(32,))

    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.01 * p.grad

print(loss.item())

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,100), generator = g)
W1 = torch.randn((300,500), generator = g)
b1 = torch.randn(500 , generator = g)
W2 = torch.randn((500,27), generator = g)
b2 = torch.randn(27 , generator = g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters) # total number of params in our model

In [ ]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range(10000):

    # mini batch construction
    ix = torch.randint(0,Xtr.shape[0],(32,))

    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,300) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.001 * p.grad

print(loss.item())

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,300) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,300) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))